# 00_setup
Drive mount + symlink storage + requirements install.


In [ ]:
step = "00_setup_drive_mount"
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print(f"OK: {step}")
except Exception as e:
    print(f"ERROR: {step} :: {e}")
    raise


In [ ]:
%%bash
    set -euo pipefail
    STEP="00_setup_symlink_and_deps"
    if [ -d "/content/Dings_Trader/TraderHimSelf" ]; then
  ROOT="/content/Dings_Trader/TraderHimSelf"
elif [ -d "/content/dings-trader/projects/dings-trader/TraderHimSelf" ]; then
  ROOT="/content/dings-trader/projects/dings-trader/TraderHimSelf"
else
  echo "ERROR: Repo nicht gefunden. Erst Clone/Update-Zelle ausführen."
  exit 1
fi
    cd "$ROOT"
    mkdir -p logs/colab
    LOG="logs/colab/${STEP}.log"
    trap 'rc=$?; echo "ERROR: ${STEP} :: exit_code=${rc}" | tee -a "$LOG"; exit $rc' ERR

    echo "START: ${STEP}" | tee -a "$LOG"
    STORE="${STORE:-/content/drive/MyDrive/dings-trader-store/TraderHimSelf}"
mkdir -p "$STORE"/{data_raw,data_processed,models,logs,runs,checkpoints}

for d in data_raw data_processed models logs runs checkpoints; do
  if [ -L "$d" ]; then
    rm "$d"
  elif [ -e "$d" ]; then
    if [ -n "$(ls -A "$d" 2>/dev/null || true)" ]; then
      mv "$d" "${d}.backup.$(date +%s)"
    else
      rm -rf "$d"
    fi
  fi
  ln -s "$STORE/$d" "$d"
done

python -m pip install -r requirements.txt 2>&1 | tee -a "$LOG"

python - <<'PY2' 2>&1 | tee -a "$LOG"
import os
for d in ["data_raw","data_processed","models","logs","runs","checkpoints"]:
    print(f"{d}: islink={os.path.islink(d)} -> {os.path.realpath(d)}")
PY2
    echo "OK: ${STEP}" | tee -a "$LOG"
